In [1]:
import pandas as pd
from rectools import Columns
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset
from rectools.models import PopularModel, PureSVDModel
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

/home/bazilyq/Рабочий стол/work/RecTools/.venv/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/bazilyq/Рабочий стол/work/RecTools/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(0)
DATA_PATH = '../datasets/KION/data/'
users_df = pd.read_csv(DATA_PATH + 'users.csv',)
items_df = pd.read_csv(DATA_PATH + 'items.csv',)
interactions = pd.read_csv(DATA_PATH + 'interactions.csv')

# interactions_train = interactions.query("last_watch_dt >= '2021-07-15' and last_watch_dt < '2021-08-16'").copy()
interactions_train = interactions.query("last_watch_dt >= '2021-08-10' and last_watch_dt < '2021-08-16'").copy()
interactions_test = interactions.query("last_watch_dt >= '2021-08-16'").copy()

sample_users = np.random.choice(interactions_train['user_id'].unique(), 10000)
interactions_train = interactions_train[interactions_train['user_id'].isin(sample_users)]

interactions_test = interactions_test.loc[
    interactions_test["user_id"].isin(interactions_train["user_id"])
    & interactions_test["item_id"].isin(interactions_train["item_id"])
]

interactions_train.fillna({"watched_pct": 1}, inplace=True)
interactions_train['watched_pct'] = interactions_train['watched_pct'].astype(int)

dataset = Dataset.construct(
    interactions_df=interactions_train
    .rename(
        columns={
            "user_id": Columns.User,
            "item_id": Columns.Item,
            "last_watch_dt": Columns.Datetime,
            "watched_pct": Columns.Weight,
        }
    )
    .reindex(columns=Columns.Interactions)
)

test_users = interactions_test["user_id"].unique()
test_users.size

user_ext_to_int_map = dataset.user_id_map.to_internal.to_dict()
item_int_to_ext_map = dataset.item_id_map.to_external.to_dict()

ui_csr = dataset.get_user_item_matrix()
ui_csr.shape

(9646, 3273)

In [3]:
%%time
pure_svd_model = PureSVDModel(verbose=1)
pure_svd_model.fit(dataset)

recs = pure_svd_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print('PureSVDModel')
print(f"precision: {precision_value}\nrecall: {recall_value}", '\n')
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

PureSVDModel
precision: 0.016442411194833154
recall: 0.06239342800454374 

(3716,) (112,)
CPU times: user 1.91 s, sys: 1.72 s, total: 3.63 s
Wall time: 644 ms


In [4]:
pure_svd_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)['item_id'].unique().shape

(112,)

In [12]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
_update_item_params --- 0.010700464248657227 seconds ---
_update_user_params --- 0.008363723754882812 seconds ---
_udpate_item_features --- 0.9364006519317627 seconds ---
_update_user_features --- 2.321230888366699 seconds ---
_update_average_features --- 0.0004551410675048828 seconds ---
iteration: 1, train RMSE: 32.63250719505038
_update_item_params --- 0.0016720294952392578 seconds ---
_update_user_params --- 0.001730203628540039 seconds ---
_udpate_item_features --- 1.071181058883667 seconds ---
_update_user_features --- 2.5688424110412598 seconds ---
_update_average_features --- 0.0007374286651611328 seconds ---
iteration: 2, train RMSE: 20.975747831859152
_update_item_params --- 0.002885103225708008 seconds ---
_update_user_params --- 0.003625631332397461 seconds ---
_udpate_item_features --- 0.8813178539276123 seconds ---
_update_user_features --- 2.7865371704101562 seconds ---
_update_average_features --- 0.0004780292510986328 seconds ---
iteration: 3, train RMSE: 15.35758388

In [5]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=20, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 31.42509274509129
iteration: 2, train RMSE: 19.18725881150648
iteration: 3, train RMSE: 13.950517255640332
iteration: 4, train RMSE: 11.165711969061135
iteration: 5, train RMSE: 9.515639612871189
iteration: 6, train RMSE: 8.466328361144381
iteration: 7, train RMSE: 7.841096998363371
iteration: 8, train RMSE: 7.482268848658177
iteration: 9, train RMSE: 7.284678970206424
iteration: 10, train RMSE: 7.206825565625938
precision: 0.0021797631862217437
recall: 0.007743531223433212
(3716,) (1441,)
CPU times: user 39.4 s, sys: 7.98 s, total: 47.3 s
Wall time: 33.8 s


In [6]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=50, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 28.558472674565465
iteration: 2, train RMSE: 16.690623065099075
iteration: 3, train RMSE: 12.374375029183549
iteration: 4, train RMSE: 10.111029144835314
iteration: 5, train RMSE: 8.789850075029214
iteration: 6, train RMSE: 7.95297652739522
iteration: 7, train RMSE: 7.4354026193946305
iteration: 8, train RMSE: 7.17304696617773
iteration: 9, train RMSE: 7.053402052769508
iteration: 10, train RMSE: 7.00926106103287
precision: 0.0008880516684607103
recall: 0.0026956000583557094
(3716,) (1527,)
CPU times: user 3min 29s, sys: 28.3 s, total: 3min 57s
Wall time: 3min 12s


In [6]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 32.63250719505038
iteration: 2, train RMSE: 20.975747831859152
iteration: 3, train RMSE: 15.357583880254976
iteration: 4, train RMSE: 12.221639801814437
iteration: 5, train RMSE: 10.34485484082636
iteration: 6, train RMSE: 9.178165858001858
iteration: 7, train RMSE: 8.4633752372555
iteration: 8, train RMSE: 8.023760232116691
iteration: 9, train RMSE: 7.747666811242639
iteration: 10, train RMSE: 7.629712043685248
precision: 0.004790096878363832
recall: 0.018530246610991296
(3716,) (1262,)
CPU times: user 38 s, sys: 6.35 s, total: 44.4 s
Wall time: 33.4 s


In [7]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=20, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 32.63250719505038
iteration: 2, train RMSE: 20.975747831859152
iteration: 3, train RMSE: 15.357583880254976
iteration: 4, train RMSE: 12.221639801814437
iteration: 5, train RMSE: 10.34485484082636
iteration: 6, train RMSE: 9.178165858001858
iteration: 7, train RMSE: 8.4633752372555
iteration: 8, train RMSE: 8.023760232116691
iteration: 9, train RMSE: 7.747666811242639
iteration: 10, train RMSE: 7.629712043685248
iteration: 11, train RMSE: 7.610726905428162
iteration: 12, train RMSE: 7.651304772321509
iteration: 13, train RMSE: 7.716848942036637
iteration: 14, train RMSE: 7.818895963863033
iteration: 15, train RMSE: 7.914735871549813
iteration: 16, train RMSE: 8.047061420303132
iteration: 17, train RMSE: 8.19467085330328
iteration: 18, train RMSE: 8.307437540169147
iteration: 19, train RMSE: 8.416199893575769
iteration: 20, train RMSE: 8.515436177704867
precision: 0.004144241119483315
recall: 0.015595703410161643
(3716,) (1331,)
CPU times: user 1min 7s, sys: 

In [8]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=1., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 32.63643061584835
iteration: 2, train RMSE: 20.97752064861894
iteration: 3, train RMSE: 15.358692204617606
iteration: 4, train RMSE: 12.222319090980495
iteration: 5, train RMSE: 10.345195727230246
iteration: 6, train RMSE: 9.179100990780988
iteration: 7, train RMSE: 8.463469465429258
iteration: 8, train RMSE: 8.0254584501313
iteration: 9, train RMSE: 7.754780434120566
iteration: 10, train RMSE: 7.638859266426678
precision: 0.005059203444564048
recall: 0.019409706814137652
(3716,) (1266,)
CPU times: user 35.2 s, sys: 6.55 s, total: 41.7 s
Wall time: 30.1 s


In [9]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=1., beta_item=1., seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 32.6528414471397
iteration: 2, train RMSE: 20.986038963971325
iteration: 3, train RMSE: 15.364408550399443
iteration: 4, train RMSE: 12.226379768478182
iteration: 5, train RMSE: 10.34858834578798
iteration: 6, train RMSE: 9.176700265113903
iteration: 7, train RMSE: 8.463590478698324
iteration: 8, train RMSE: 8.023022442528267
iteration: 9, train RMSE: 7.748493074484703
iteration: 10, train RMSE: 7.642072223896064
precision: 0.004843918191603875
recall: 0.018478634871383317
(3716,) (1269,)
CPU times: user 36.6 s, sys: 6.73 s, total: 43.3 s
Wall time: 31.7 s


In [12]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=0.1, beta_item=0.1, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 32.671271559386724
iteration: 2, train RMSE: 20.995375481357883
iteration: 3, train RMSE: 15.370608005714807
iteration: 4, train RMSE: 12.23062966318669
iteration: 5, train RMSE: 10.35150709583589
iteration: 6, train RMSE: 9.177147887208971
iteration: 7, train RMSE: 8.463854692830614
iteration: 8, train RMSE: 8.018383859423915
iteration: 9, train RMSE: 7.741312539211905
iteration: 10, train RMSE: 7.617317832599062
precision: 0.004601722282023682
recall: 0.018116713836537568
(3716,) (1287,)
CPU times: user 37 s, sys: 5.21 s, total: 42.2 s
Wall time: 30.9 s


In [15]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2., seed=0, mu0_user=10., mu0_item=0.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 29.21672151897565
iteration: 2, train RMSE: 19.575229969230563
iteration: 3, train RMSE: 14.503785019263214
iteration: 4, train RMSE: 11.628951407173313
iteration: 5, train RMSE: 9.91941845521111
iteration: 6, train RMSE: 8.874502745122413
iteration: 7, train RMSE: 8.253395739473696
iteration: 8, train RMSE: 7.917814642405132
iteration: 9, train RMSE: 7.767509658805079
iteration: 10, train RMSE: 7.734772266094451
precision: 0.00842303552206674
recall: 0.03182212208887856
(3716,) (1248,)
CPU times: user 34.3 s, sys: 6.46 s, total: 40.7 s
Wall time: 29.4 s


In [17]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2., seed=0, mu0_user=10., mu0_item=10.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 26.074060504539908
iteration: 2, train RMSE: 18.016527656260216
iteration: 3, train RMSE: 13.422139779497805
iteration: 4, train RMSE: 10.761811371779006
iteration: 5, train RMSE: 9.13516013297758
iteration: 6, train RMSE: 8.11295700712859
iteration: 7, train RMSE: 7.492409825865953
iteration: 8, train RMSE: 7.131306630856232
iteration: 9, train RMSE: 6.90914108679794
iteration: 10, train RMSE: 6.801809913747666
precision: 0.005086114101184068
recall: 0.01858562489256749
(3716,) (1232,)
CPU times: user 33.6 s, sys: 6.41 s, total: 40 s
Wall time: 28.7 s


In [18]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2., seed=0, mu0_user=10., mu0_item=5.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 26.78916195958925
iteration: 2, train RMSE: 18.38638882251181
iteration: 3, train RMSE: 13.678975625034788
iteration: 4, train RMSE: 10.968019971674012
iteration: 5, train RMSE: 9.339069873753276
iteration: 6, train RMSE: 8.324035457555842
iteration: 7, train RMSE: 7.743784485280171
iteration: 8, train RMSE: 7.434481827282865
iteration: 9, train RMSE: 7.285398695082089
iteration: 10, train RMSE: 7.212279540959219
precision: 0.0060279870828848225
recall: 0.022403373686444624
(3716,) (1246,)
CPU times: user 34.5 s, sys: 6.37 s, total: 40.8 s
Wall time: 29.6 s


In [19]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=50, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2., seed=0, mu0_user=10., mu0_item=0.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 29.21672151897565
iteration: 2, train RMSE: 19.575229969230563
iteration: 3, train RMSE: 14.503785019263214
iteration: 4, train RMSE: 11.628951407173313
iteration: 5, train RMSE: 9.91941845521111
iteration: 6, train RMSE: 8.874502745122413
iteration: 7, train RMSE: 8.253395739473696
iteration: 8, train RMSE: 7.917814642405132
iteration: 9, train RMSE: 7.767509658805079
iteration: 10, train RMSE: 7.734772266094451
iteration: 11, train RMSE: 7.760323759853407
iteration: 12, train RMSE: 7.840583162333499
iteration: 13, train RMSE: 7.942479296640077
iteration: 14, train RMSE: 8.057678272950493
iteration: 15, train RMSE: 8.182467291343647
iteration: 16, train RMSE: 8.311039361132009
iteration: 17, train RMSE: 8.4487231995727
iteration: 18, train RMSE: 8.573616081250384
iteration: 19, train RMSE: 8.708680393407546
iteration: 20, train RMSE: 8.823642268576812
iteration: 21, train RMSE: 8.926980207367524
iteration: 22, train RMSE: 9.030519714006468
iteration: 23, tr

In [22]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=20, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2., seed=0, mu0_user=10., mu0_item=0.)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs , interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
iteration: 1, train RMSE: 27.736912571123167
iteration: 2, train RMSE: 17.62599258970381
iteration: 3, train RMSE: 12.984050296392224
iteration: 4, train RMSE: 10.483048201686477
iteration: 5, train RMSE: 9.007825364981574
iteration: 6, train RMSE: 8.092197836682338
iteration: 7, train RMSE: 7.574130929892149
iteration: 8, train RMSE: 7.292846043741413
iteration: 9, train RMSE: 7.153653870730035
iteration: 10, train RMSE: 7.117144386024365
precision: 0.006001076426264801
recall: 0.022731263131883984
(3716,) (1378,)
CPU times: user 37.1 s, sys: 7.23 s, total: 44.4 s
Wall time: 32.1 s


In [8]:
%%time
from rectools.models import BayesProbMF_V2

bpmf_model = BayesProbMF_V2(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

iteration: 1, train RMSE: 32.63250719505038
iteration: 2, train RMSE: 20.975747831859145
iteration: 3, train RMSE: 15.35758388025496
iteration: 4, train RMSE: 12.221639801814359
iteration: 5, train RMSE: 10.344854840826153
iteration: 6, train RMSE: 9.178165858001298
iteration: 7, train RMSE: 8.463375237252245
iteration: 8, train RMSE: 8.023760232113574
iteration: 9, train RMSE: 7.747666811236417
iteration: 10, train RMSE: 7.629712043681804
precision: 0.004790096878363832
recall: 0.018530246610991296
(3716,) (1262,)
CPU times: user 7min 27s, sys: 8min 25s, total: 15min 52s
Wall time: 2min 13s


In [5]:
%%time
# numba version
from rectools.models import BayesProbMF_V3

bpmf_model = BayesProbMF_V3(verbose=1, n_iters=10, n_feature=10, max_rating=100., min_rating=0., beta=2., beta_user=2., beta_item=2, seed=0)
bpmf_model.fit(dataset)

recs = bpmf_model.recommend(users=test_users, dataset=dataset, k=10, filter_viewed=False)

precision = Precision(k=10)
recall = Recall(k=10)

precision_value = precision.calc(reco=recs, interactions=interactions_test)
recall_value = recall.calc(reco=recs, interactions=interactions_test)
print(f"precision: {precision_value}\nrecall: {recall_value}")
print(recs['user_id'].unique().shape, recs['item_id'].unique().shape)

0
_update_item_params --- 0.0016851425170898438 seconds ---
_update_user_params --- 0.0013153553009033203 seconds ---


_udpate_item_features --- 2.236502170562744 seconds ---
_update_user_features --- 0.25388550758361816 seconds ---
_update_average_features --- 0.00035190582275390625 seconds ---
iteration: 1, train RMSE: 35.58309304232626
_update_item_params --- 0.001379251480102539 seconds ---
_update_user_params --- 0.0011057853698730469 seconds ---
_udpate_item_features --- 0.3350207805633545 seconds ---
_update_user_features --- 0.24868297576904297 seconds ---
_update_average_features --- 0.0003561973571777344 seconds ---
iteration: 2, train RMSE: 22.60438741735845
_update_item_params --- 0.0014882087707519531 seconds ---
_update_user_params --- 0.0015611648559570312 seconds ---
_udpate_item_features --- 0.32807040214538574 seconds ---
_update_user_features --- 0.2336585521697998 seconds ---
_update_average_features --- 0.0002827644348144531 seconds ---
iteration: 3, train RMSE: 16.36181742125372
_update_item_params --- 0.0014884471893310547 seconds ---
_update_user_params --- 0.0012271404266357422